In [138]:
import torch
import cv2
import numpy as np
from deepface import DeepFace

In [139]:
rojo = [0, 0, 255]
azul = [255, 0, 0]
amarillo = [0, 255, 255]
lila = [255, 0, 255]
naranja =  [0, 128, 255]
verde =  [0, 128, 0]

lower_range = np.array([0, 0, 100], dtype=np.uint8)
upper_range = np.array([100, 100, 255], dtype=np.uint8)

In [140]:
def paint_emo_color(img_url, color):
    img = cv2.imread(img_url)
    img_copia = img.copy()
    model = YOLO('yolov8n-seg.pt') 

    results = model(img, stream=True)

    for result in results:
        masks = result.masks.data
        boxes = result.boxes.data
        # extract classes
        clss = boxes[:, 5]
        # get indices of results where class is 0 (people in COCO)
        people_indices = torch.where(clss == 0)
        # use these indices to extract the relevant masks
        people_masks = masks[people_indices]
        people_boxes = boxes[people_indices]

        people_mask = torch.any(people_masks, dim=0).int() * 255
        # Resize the mask to match the original image size
        print("Original Image Shape:", img_copia.shape)
        print("Resized Mask Shape:", people_mask.shape)
        people_mask = cv2.resize(people_mask.numpy().astype(np.uint8), (img_copia.shape[1], img_copia.shape[0]))

        # Make the mask yellow and transparent
        yellow_color = np.array(color)  # Amarillo
        img_copia[people_mask > 0] = (img_copia[people_mask > 0] * 0.7 + yellow_color * 0.3).astype(int)
        # cv2.imwrite('merged_segs2.jpg', img_copia)
        return img_copia

#------------------------------------------------------------------------------------------------------------------

In [141]:

def analizar_y_modificar(img_path, nombre, frame):
    # Analizar la imagen para obtener la emoción
    result = DeepFace.analyze(img_path=img_path, enforce_detection=False, actions=['emotion'])
    
    # Imprimir los resultados para entender la estructura
    print(result)
    
    # Obtener la emoción detectada (asumiendo que result es una lista)
    emocion_detectada = result[0]['dominant_emotion']

    # Cargar la imagen original
    imagen = cv2.imread(img_path)
    img_copia = imagen.copy()

    # Definir un rango de colores para "angry" (por ejemplo, rojo)
    if emocion_detectada.lower() == 'angry':
    
        mask = cv2.inRange(imagen, lower_range, upper_range)

        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path, rojo)
        img_copia[mask > 0] = rojo  


    elif emocion_detectada.lower() == 'happy':
        # Crear una máscara para los píxeles en el rango de colores
        mask = cv2.inRange(imagen, lower_range, upper_range)
        
        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path, amarillo)
        img_copia[mask > 0] = amarillo 

    elif emocion_detectada.lower() == 'sad':
        # Crear una máscara para los píxeles en el rango de colores
        mask = cv2.inRange(imagen, lower_range, upper_range)
        
        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path, azul)
        img_copia[mask > 0] = azul 

    elif emocion_detectada.lower() == 'fear':
        # Crear una máscara para los píxeles en el rango de colores
        mask = cv2.inRange(imagen, lower_range, upper_range)
        
        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path,  lila)
        img_copia[mask > 0] =  lila  

    elif emocion_detectada.lower() == 'surprise':
        # Crear una máscara para los píxeles en el rango de colores
        mask = cv2.inRange(imagen, lower_range, upper_range)
        
        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path,  naranja)
        img_copia[mask > 0] =  naranja  
        print("surprise")

    elif emocion_detectada.lower() == 'disgust':
        # Crear una máscara para los píxeles en el rango de colores
        mask = cv2.inRange(imagen, lower_range, upper_range)
        
        # Aplicar la máscara para cambiar el color de la cara a rojo
        img_copia = paint_emo_color(img_path,  verde)
        img_copia[mask > 0] =  verde  
        print("asqueado")

    # Guardar la imagen modificada
    cv2.imwrite(nombre + "_modificado.jpg", imagen)
    height, width, _ = img_copia.shape
    
    # Calcular nuevas dimensiones manteniendo la proporción
    new_width = 400
    new_height = int((new_width / width) * height)
    
    # Mostrar la imagen en una ventana redimensionada
    cv2.namedWindow('Imagen Resultante', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Imagen Resultante', new_width, new_height)
    cv2.imshow('Imagen Resultante', img_copia)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [142]:
analizar_y_modificar("C:/Users/Raul/Downloads/PR6_Minerva/triste.jpg", "nombre_salida", None)

Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  6.43it/s]


[{'emotion': {'angry': 1.1823140789457607, 'disgust': 7.319572300453265e-05, 'fear': 38.410066867474725, 'happy': 0.00048941222491896, 'sad': 60.25357009491742, 'surprise': 0.0013842904906785968, 'neutral': 0.1521070927706118}, 'dominant_emotion': 'sad', 'region': {'x': 73, 'y': 25, 'w': 115, 'h': 115}}]

Original Image Shape: (183, 275, 3)
Resized Mask Shape: torch.Size([448, 640])
